In [1]:
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt 
import seaborn as sns 
%matplotlib inline
sns.set()
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score, StratifiedKFold, learning_curve, train_test_split
from sklearn.preprocessing import OneHotEncoder, LabelEncoder, MinMaxScaler, StandardScaler
from sklearn.feature_selection import RFECV
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.utils import shuffle
# loading data
df_train = pd.read_csv("2-1.csv")
df_test = pd.read_csv("2-2.csv")
df_data = df_train.append(df_test)

In [2]:
df_data = shuffle(df_data)
df_data = df_data.reset_index()
del df_data['index']


In [3]:
#cut data
df_train = df_data.loc[:159]
df_test = df_data.loc[160:]

In [4]:
print(df_train.shape)
print(df_test.shape)

(160, 202)
(22, 202)


In [5]:
df_data

,代號,一個月累計漲跌幅,1,2,3,4,5,6,7,8,...,193,194,195,196,197,198,199,200,201,202
0,3014,4.70,16.10,118.00,16.10,24.2,17.8,13.90,17.50,10.80,...,2.90,4567,7517↘,5555↘,4774↗,0.0052,0.0213,0.0544,0.1460,0.3580
1,6187,14.10,8.33,49.30,8.33,24.2,17.9,17.20,22.70,12.30,...,6.78,1536,4833↘,5122↘,5709↘,-0.0491,-0.0476,0.0263,0.2250,0.3510
2,2456,16.60,25.50,264.00,25.50,48.0,18.9,13.60,20.40,9.87,...,2.71,8314,14896↗,10575↗,7122↗,-0.0038,0.0488,0.0926,0.0847,0.0539
3,1101,3.45,567.00,2542.00,547.00,69.6,58.5,13.50,15.50,11.30,...,0.39,32520,33153↗,24103↗,21549↗,-0.0056,0.0119,0.0219,0.0559,0.0902
4,8101,-2.07,20.80,7.87,20.80,21.0,16.1,-2.19,-3.48,-1.57,...,0.10,254,272↗,236↗,218↗,-0.0042,-0.0071,-0.0142,-0.0479,-0.1440
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
177,5205,-27.40,1.80,1.39,1.80,30.7,19.2,-18.60,-27.50,-12.00,...,0.01,5,2↘,2.88↗,2.76↗,-0.0082,0.0096,0.0058,-0.0703,-0.0179
178,2227,4.07,30.00,843.00,30.00,16.8,15.6,12.80,15.40,10.50,...,0.06,583,310↗,282↗,184↗,0.0061,0.0108,0.0230,0.0392,0.0984
179,3339,-1.16,9.62,4.11,9.62,20.2,14.1,-2.72,-4.47,-1.80,...,0.35,54,181↘,162↘,337↘,0.0196,0.0991,0.0260,0.2730,0.2560
180,8444,15.80,8.48,61.00,8.48,9.6,4.8,37.30,49.10,25.80,...,0.04,35,26.4↗,22.6↗,29.6↘,0.0053,0.0161,0.0193,0.1930,0.3280


In [6]:
#create rise_fall
df_data['rise_fall'] = (df_data['一個月累計漲跌幅']).map(lambda x : 1 if x >= 0 else 0)

In [7]:
for a in df_data:
    if type(df_data[a][0]) == str :
        
        print(f"{a} {df_data[a][0]}")

14 20M07
15 20Q2
58 4.21↗
59 3.57↗
60 3.49↗
73 20M07
74 20Q2
75 1高
76 1高
77 2高
78 連5增
159 20W32
170 20M08
183 73.12↗
184 71.97↗
185 69.71↗
186 64.15↗
187 54.14↗
188 47.37↗
195 7517↘
196 5555↘
197 4774↗


In [8]:
#delete the feature which is not important
del df_data['14']
del df_data['15']
del df_data['73']
del df_data['74']
del df_data['159']
del df_data['170']

In [9]:
for a in df_data:
    if type(df_data[a][0]) == str :
        
        print(f"{a} {df_data[a][0]}")

58 4.21↗
59 3.57↗
60 3.49↗
75 1高
76 1高
77 2高
78 連5增
183 73.12↗
184 71.97↗
185 69.71↗
186 64.15↗
187 54.14↗
188 47.37↗
195 7517↘
196 5555↘
197 4774↗


In [10]:
df_data['58'][0][-1:]

'↗'

In [11]:
#change str to float or int
df_data['new_58'] = (df_data['58']).map(lambda x : float(x[:-1]) if x[-1:]=='↗' else float(x[:-1])*-1)
del df_data['58']

In [12]:
df_data['new_59'] = (df_data['59']).map(lambda x : float(x[:-1]) if x[-1:]=='↗' else float(x[:-1])*-1)
del df_data['59']
df_data['new_60'] = (df_data['60']).map(lambda x : float(x[:-1]) if x[-1:]=='↗' else float(x[:-1])*-1)
del df_data['60']
df_data['new_183'] = (df_data['183']).map(lambda x : float(x[:-1]) if x[-1:]=='↗' else float(x[:-1])*-1)
del df_data['183']
df_data['new_184'] = (df_data['184']).map(lambda x : float(x[:-1]) if x[-1:]=='↗' else float(x[:-1])*-1)
del df_data['184']
df_data['new_185'] = (df_data['185']).map(lambda x : float(x[:-1]) if x[-1:]=='↗' else float(x[:-1])*-1)
del df_data['185']
df_data['new_186'] = (df_data['186']).map(lambda x : float(x[:-1]) if x[-1:]=='↗' else float(x[:-1])*-1)
del df_data['186']
df_data['new_187'] = (df_data['187']).map(lambda x : float(x[:-1]) if x[-1:]=='↗' else float(x[:-1])*-1)
del df_data['187']
df_data['new_188'] = (df_data['188']).map(lambda x : float(x[:-1]) if x[-1:]=='↗' else float(x[:-1])*-1)
del df_data['188']
df_data['new_195'] = (df_data['195']).map(lambda x : float(x[:-1]) if x[-1:]=='↗' else float(x[:-1])*-1)
del df_data['195']
df_data['new_196'] = (df_data['196']).map(lambda x : float(x[:-1]) if x[-1:]=='↗' else float(x[:-1])*-1)
del df_data['196']
df_data['new_197'] = (df_data['197']).map(lambda x : float(x[:-1]) if x[-1:]=='↗' else float(x[:-1])*-1)
del df_data['197']

In [13]:
for a in df_data:
    if type(df_data[a][0]) == str :
        
        print(f"{a} {df_data[a][0]}")
        


75 1高
76 1高
77 2高
78 連5增


In [14]:
testdata = df_data.drop(labels=['75','76','77','78'],axis=1)

In [33]:
#plt.figure(figsize=(100,100))
#stock_corr = sns.heatmap(testdata.corr(),annot=True)

In [30]:
S_C = testdata.corr()

In [32]:
S_C['一個月累計漲跌幅']

代號          0.028036
一個月累計漲跌幅    1.000000
1           0.043806
2           0.115353
3           0.043972
              ...   
new_187     0.159200
new_188     0.089813
new_195     0.215237
new_196     0.235496
new_197     0.228004
Name: 一個月累計漲跌幅, Length: 193, dtype: float64

In [140]:
def get_index(corrvalue):
    output = []
    for index, value in S_C['一個月累計漲跌幅'].items():
        if abs(value) >=corrvalue:
            output.append(index)
    return output


In [141]:
#find NaN
x =testdata.isnull().any()
IND = []
for index, value in x.items():
    if value==True:
        #print(f"Index : {index}, Value : {value}")
        IND.append(index)

In [142]:
for I in IND:
    testdata[I] = testdata[I].fillna(testdata[I].median())#把NaN用中位數取代

In [171]:
val_arr = {0.1,0.2,0.3,0.4,0.5,0.6}

In [172]:
test_index = get_index(0.7)
corr_data = testdata[test_index]
test_index

['一個月累計漲跌幅', '99', '200', '201']

In [173]:
def run_sklearn(val_arr,data):
    for x in val_arr:
        corr_index = get_index(x)
        corr_data = data[corr_index]
        df_train = corr_data.loc[:150]
        df_test = corr_data.loc[151:]
        X = df_train.drop(labels=['rise_fall','一個月累計漲跌幅'],axis=1)
        Y = df_train['rise_fall']
        Base_Model = RandomForestClassifier(random_state=2,n_estimators=250,min_samples_split=22,oob_score=True)
        Base_Model.fit(X, Y)
        print('Base oob score :%.5f' %(Base_Model.oob_score_),'   LB_Public : 0.77751')
        

In [174]:
run_sklearn(val_arr,testdata)

Base oob score :0.97351    LB_Public : 0.77751
Base oob score :0.99338    LB_Public : 0.77751
Base oob score :0.99338    LB_Public : 0.77751
Base oob score :0.98675    LB_Public : 0.77751
Base oob score :0.99338    LB_Public : 0.77751
Base oob score :0.98675    LB_Public : 0.77751


In [175]:
#cut data
df_train = testdata.loc[:150]
df_test = testdata.loc[151:]

In [413]:
X = df_train.drop(labels=['代號','rise_fall','一個月累計漲跌幅'],axis=1)
Y = df_train['rise_fall']

In [414]:
Base_Model = RandomForestClassifier(random_state=2,n_estimators=250,min_samples_split=22,oob_score=True)
Base_Model.fit(X, Y)
print('Base oob score :%.5f' %(Base_Model.oob_score_),'   LB_Public : 0.77751')

Base oob score :0.95364    LB_Public : 0.77751


In [415]:
X_Submit = df_test.drop(labels=['代號','rise_fall','一個月累計漲跌幅'],axis=1)

Base_pred = Base_Model.predict(X_Submit)

submit = pd.DataFrame({"代號": df_test['代號'],
                      "rise_fall":Base_pred.astype(int)})


### 上面是對漲跌做預測,再來是對漲跌幅度做預測
---

In [73]:
updown = testdata.drop(labels=['rise_fall'],axis=1)

In [74]:
max(updown['一個月累計漲跌幅'])

59.8

In [75]:
min(updown['一個月累計漲跌幅'])

-35.2

In [76]:
updown['一個月累計漲跌幅'] = (updown['一個月累計漲跌幅']).map(lambda x : int(x/7))

In [77]:
to_log_index = []
for x in updown:
    if max(updown[x])+abs(min(updown[x])) >=20:
        to_log_index.append(x)

    

In [78]:
for x in to_log_index:
    updown[x] = (updown[x]).map(lambda x : np.log10(x) if x > 0 else (-1*np.log10(abs(x)) if x<0 else 0))

In [91]:
u_p = updown.corr()

In [148]:
def up_index(corrvalue):
    output = []
    for index, value in u_p['一個月累計漲跌幅'].items():
        if abs(value) >=corrvalue:
            output.append(index)
    return output

In [155]:
def updown_sklearn(val_arr,data):
    for x in val_arr:
        corr_index = up_index(x)
        corr_data = data[corr_index]
        df_train = corr_data.loc[:150]
        df_test = corr_data.loc[151:]
        X = df_train.drop(labels=['一個月累計漲跌幅'],axis=1)
        Y = df_train['一個月累計漲跌幅']
        Base_Model = RandomForestClassifier(random_state=2,n_estimators=950,min_samples_split=15,oob_score=True)
        Base_Model.fit(X, Y.astype('int'))
        print('Base oob score :%.5f' %(Base_Model.oob_score_),'   LB_Public : 0.77751', '%f' % x)
    return Base_Model

In [176]:
#final_model = updown_sklearn(val_arr,updown)

Base oob score :0.61589    LB_Public : 0.77751 0.100000
Base oob score :0.78808    LB_Public : 0.77751 0.400000
Base oob score :0.76821    LB_Public : 0.77751 0.300000
Base oob score :0.70199    LB_Public : 0.77751 0.200000
Base oob score :0.84106    LB_Public : 0.77751 0.500000
Base oob score :0.84106    LB_Public : 0.77751 0.600000


In [185]:
final_model = updown_sklearn({0.5},updown)

Base oob score :0.84106    LB_Public : 0.77751 0.500000


In [195]:
corr_index = up_index(0.5)
corr_data = updown[corr_index]
df_train = corr_data.loc[:150]
df_test = corr_data.loc[151:]

In [187]:
X = df_train.drop(labels=['一個月累計漲跌幅'],axis=1)
Y = df_train['一個月累計漲跌幅']

In [188]:
ud_Model = RandomForestClassifier(random_state=2,n_estimators=950,min_samples_split=15,oob_score=True)
ud_Model.fit(X, Y.astype('int'))
print('Base oob score :%.5f' %(ud_Model.oob_score_),'   LB_Public : 0.77751')

Base oob score :0.84106    LB_Public : 0.77751


In [189]:
test_x = df_train.drop(labels=['一個月累計漲跌幅'],axis=1)
test_y = df_train['一個月累計漲跌幅']

In [190]:
test_y_predicted = final_model.predict(test_x)
print(test_y_predicted)


[ 0  1  2  0  0  0  0  0  3 -1  1  0  0  3  0  0  0  0 -1  0 -1  0  4  2
  0  0  0  0  0  1  0  0  0  0 -1  0  0  2  0  0  0  0  0  1  2  4  1  0
  4  0  1  1  0  4  4  0  1  4 -1  3  0  1  0  0 -2  1 -1  0  2  0  0  2
  4  0  0  0  1 -1  0  0  3  4  1  1  3 -1  0  0 -1  0  0  0  0  0  0  2
  1  0 -1  0  4 -1  2  1  0  3  0  1  0  0  1  0  1 -1  3  1  0  0  4  0
  0  3  0  0  4  0  2  2  3  0  0  0  0  0  0  0  2  0 -1  3  1  3  1 -1
  0  2  0 -1  0  0  0]


In [192]:
ans = []
for x in test_y:
    ans.append(x)
print(ans)


[0, 2, 2, 0, 0, 0, 0, -4, 3, -3, 1, 0, 0, 3, 0, 0, 0, 0, -1, 0, -1, 0, 4, 2, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, -1, 0, 0, 2, 0, 0, 0, 0, 0, 1, 2, 4, 1, 0, 4, 0, 1, 1, 0, 4, 8, 0, 1, 4, -1, 3, 0, 1, 0, 0, -2, 1, -1, 0, 2, 0, 0, 2, 7, 0, -2, -2, 1, -1, 0, 0, 3, 7, 5, 1, 3, -1, 0, 0, -1, 0, 0, -1, 0, 0, 0, 2, 1, 0, -3, 0, 4, -1, 2, 1, 0, 3, 0, 1, 0, 0, 1, 0, 1, -1, 3, 4, 0, 0, 8, 0, -5, 3, 0, 0, 4, 0, 2, 2, 3, 0, 0, 0, 0, 0, 0, 0, 2, 0, -4, 3, 1, 5, 1, -1, 0, 2, 0, -1, 0, 0, 0]


In [193]:
SS=0
for s in range(0,151):
    if test_y_predicted[s] == ans[s]:
        SS +=1
SS/151

0.8940397350993378